<a href="https://colab.research.google.com/github/smriti-mittal-au3/lw/blob/main/Make_your_own_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# saves a input.txt locally
# is 1MB

!wget https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt

--2025-08-05 04:18:50--  https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-08-05 04:18:50 (12.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
# 'r' = read
# encoding = utf-8 ? = text files are stored as bytes. so, encoding tells python how to convert it to chars
# ascii = is just plain english. utf-16 = is old ? ... hmm
# read() = read as string
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
# == 1M chars roughly
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(list(set(text)))
print(set(text))

['Z', "'", 'K', 'w', 'l', '?', 'p', 'o', '-', 'L', 'z', 'i', '.', 'x', 'n', '&', 'q', ',', 's', 'A', 'V', '$', ' ', '!', 't', 'P', 'I', 'G', 'Y', 'b', 'R', 'E', 'k', 'T', 'X', '\n', 'M', ';', 'h', 'U', 'j', 'e', 'u', 'f', 'd', 'y', 'O', 'c', 'D', '3', 'Q', 'C', 'a', 'F', 'r', 'm', 'H', 'B', 'J', 'W', 'S', 'N', ':', 'g', 'v']
{'Z', "'", 'K', 'w', 'l', '?', 'p', 'o', '-', 'L', 'z', 'i', '.', 'x', 'n', '&', 'q', ',', 's', 'A', 'V', '$', ' ', '!', 't', 'P', 'I', 'G', 'Y', 'b', 'R', 'E', 'k', 'T', 'X', '\n', 'M', ';', 'h', 'U', 'j', 'e', 'u', 'f', 'd', 'y', 'O', 'c', 'D', '3', 'Q', 'C', 'a', 'F', 'r', 'm', 'H', 'B', 'J', 'W', 'S', 'N', ':', 'g', 'v'}


In [9]:
print(sorted(list(set(text))))


['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab = ''.join(chars)
# why is \n new line not in vocabulary?
print(vocab)
vocab_size = len(chars)
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [16]:
# just assign numbers from 0, 1, 2, 3 ... to chars
# so our tokenizer is very simple = converting chars to int or tokens
# google uses https://github.com/google/sentencepiece - sub-word unit level. so, not at word level nor at char level
# openai / gpt uses https://github.com/openai/tiktoken
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# lambda == def any_func():
encode = lambda s: [stoi[c] for c in s]          # encoder: take a string, output a list of integers
decode = lambda i: ''.join([itos[c] for c in i]) # decoder: take a list of integers, output a string

print(stoi)
print(itos)
print(encode("hii there"))
print(decode(encode("hii there")))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [18]:
# didn't have to pip install tiktoken
import tiktoken

# encoding_for_model vs get_encoding ?
enc4o = tiktoken.encoding_for_model("gpt-4o")
print(enc4o) # o200k_base

enc2 = tiktoken.encoding_for_model("gpt2")
print(enc2) # gpt2

enc2_vocab = tiktoken.get_encoding("gpt2")
# 50,257
print(enc2_vocab.n_vocab)

enc4o_vocab = tiktoken.get_encoding("o200k_base")
# 2,00,019
print(enc4o_vocab.n_vocab)

# [71, 4178, 612]
# [71, 3573, 1354]
# [46, 47, 47, 1, 58, 46, 43, 56, 43]
print(enc2_vocab.encode("hii there"))
print(enc4o_vocab.encode("hii there"))
print(encode("hii there")) # our vocab is just 65 int, and bigger sequence


<Encoding 'o200k_base'>
<Encoding 'gpt2'>
50257
200019
[71, 4178, 612]
[71, 3573, 1354]
[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [21]:
# let's tokenize the whole of shakespeare
# why not just encode and be done with it? why store in torch tensor? what is that? how is it able to store?
data_ = encode(text)
# [18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
print(data_[:10])

import torch
# datatype float32 (32-bit floating point) vs long (64-bit integer)
# 32 bit vs 64 bit?
data = torch.tensor(data_, dtype=torch.long)
print(data.shape, data.dtype)
# float32: tensor([18., 47., 56., 57., 58.,  1., 15., 47., 58., 47.])
# long: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
print(data[:10])


[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [23]:
# Let's now split up the data into train and validation sets
print(len(data)) # 1115394
print(0.9*len(data)) # 1003854.6
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] # training data
val_data = data[n:]

# we don't send ALL data to transformer at once to train it on
# we send in chunks

1115394
1003854.6


In [24]:
# chunk size  =8
block_size = 8
train_data[:block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [26]:
# if 18, then next is 47
# if 18, 47, then next is 56
# if 18, 47, 56 then next is 57
print(range(1, block_size), "range")
for i in range(1, block_size+1):
    context = train_data[:i] # All previous values
    target = train_data[i]
    print(f"Input: {context} → Target: {target}")

# these are the 8 examples hidden in the chunk of 8
# we train on all 8 examples
# we go from 1 to 8 one by one, to make transformer get used to see a context as
# little as 1 and as big as 8

# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f"when input is {context} the target: {target}")

range(1, 8) range
Input: tensor([18]) → Target: 47
Input: tensor([18, 47]) → Target: 56
Input: tensor([18, 47, 56]) → Target: 57
Input: tensor([18, 47, 56, 57]) → Target: 58
Input: tensor([18, 47, 56, 57, 58]) → Target: 1
Input: tensor([18, 47, 56, 57, 58,  1]) → Target: 15
Input: tensor([18, 47, 56, 57, 58,  1, 15]) → Target: 47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) → Target: 58


In [34]:
# that was time dimension
# now onto batch dimension. use GPU i.e.? for parallel processing. each batch is unaware of others
# process chunks parallelly

# Set random seed for torch's randint() so that same random numbers are generated everytime
torch.manual_seed(1337)

batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch():

    # (batch_size,) = How many such sequences (contexts) you want to process in parallel
    # len(data) - block_size = max starting index
    # ix = will give 4 index b/w 0 to length of my train_data
    ix = torch.randint(len(train_data) - block_size, (batch_size,))
    print(ix, "random indices") # tensor([ 76049, 234249, 934904, 560986])

    # stack is just used to make a 2D tensor
    # context stack - 32 independent examples
    x = torch.stack([train_data[i:i+block_size] for i in ix])
    # targets stack
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch()
print(xb)
print(yb)
print ("......")

# diff b/w range and enumerate and just simple for i in ix ?

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

tensor([ 76049, 234249, 934904, 560986]) random indices
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
......
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44,

In [35]:
# input to the transformer is a 2D tensor. which will be xb. 8 * 4 = 32 examples.
# 1 row has 8 examples, [1], [1,2], [1,2,3] ... etc
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
